In [1]:
import os
import hail as hl
%run {os.environ['NB_DIR']}/nb.py
%run $BENCHMARK_METHOD_DIR/common.py
sample_rate = .05
ds_name = DATASET_HM
ds_config = DATASET_CONFIG[ds_name]
ds_export_path = dataset_path(ds_name, sr=sample_rate)
MIN_LD = .1
hail_init()

Running on Apache Spark version 2.4.4
SparkUI available at http://2e4e0c6972f9:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.32-a5876a0a2853
LOGGING: writing to /home/eczech/repos/gwas-analysis/notebooks/benchmark/method/ld_prune/lsh/hail-20200221-1251-0.2.32-a5876a0a2853.log


In [2]:
mt = hl.import_plink(
    *plink_files(osp.dirname(ds_export_path), osp.basename(ds_export_path)),
    skip_invalid_loci=False,
    reference_genome=ds_config['reference_genome']
)
mt.count()

2020-02-20 22:34:34 Hail: INFO: Found 165 samples in fam file.
2020-02-20 22:34:34 Hail: INFO: Found 72732 variants in bim file.
2020-02-20 22:34:36 Hail: INFO: Coerced sorted dataset


(72732, 165)

In [3]:
# Run LD calculations with no effective radius limit (longest known genome < 150B bp)
ld = hl.ld_matrix(mt.GT.n_alt_alleles(), mt.locus, radius=150e9)
ld.shape

2020-02-20 22:34:37 Hail: INFO: Coerced sorted dataset
2020-02-20 22:34:39 Hail: INFO: Wrote all 18 blocks of 72732 x 165 matrix with block size 4096.
2020-02-20 22:34:39 Hail: INFO: Coerced sorted dataset


(72732, 72732)

In [9]:
%%time
lde = ld.entries()
lde = lde.filter(lde.entry >= MIN_LD)
lde.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    'i': int64 
    'j': int64 
    'entry': float64 
----------------------------------------
Key: ['i', 'j']
----------------------------------------


In [17]:
%%time
path = dataset_path(ds_name, sr=sample_rate, minld=MIN_LD) + '.parquet'
lde.to_spark().write.parquet(path, mode='overwrite')
!du -ch $path

268M	/tmp/benchmark_datasets/hapmap-0.05-ld-0.1
268M	total
